In [ ]:
import os, sys, re, cv2
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import pandas as pd
import glob
import scipy
from PIL import Image
import matplotlib.backends.backend_pdf


from scipy.signal import correlate2d, fftconvolve
from numpy import flip, arange, reshape
from matplotlib.patches import Rectangle
from scipy import ndimage
from sklearn.manifold import TSNE
from sklearn import metrics
from sklearn.cluster import DBSCAN, HDBSCAN
plt.rcParams['image.cmap'] = 'gray'


# Functions

## FFT utils

In [ ]:
def iFFT(ft):
    ft = np.fft.ifftshift(ft)
    ift = np.fft.ifft2(ft)
    ift = np.fft.fftshift(ift)
    return(ift.real)   # take only the real part since computational error leads to noise in imaginary part

def FFT(img):
    if len(img.shape)==3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ft = np.fft.ifftshift(img)
    ft = np.fft.fft2(ft)
    return(np.fft.fftshift(ft))

def FFTplot(img, size=10, plot=1, **plot_args):
    if len(img.shape)==3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    ft = FFT(img)

    if plot == 1:
        plt.subplot(121)
        plt.imshow(img)
        plt.subplot(122)
        plt.imshow(np.real(ft), cmap='PuOr', **plot_args)
    return ft

def iFFTplot(ft, xylen=10, plot=1, angle="h", **plot_args):
    ift = iFFT(ft)
    a, _ =ft.shape
    xyrange = [a//2-xylen, a//2+xylen]
    if plot == 1:
        if angle == "h":
            plt.figure()
            ax=plt.subplot(121)
            ax.set_xlim(xyrange)
            ax.set_ylim(xyrange)
            plt.imshow(abs(ft), **plot_args)
            plt.subplot(122)
            plt.imshow(ift)
        elif angle == "v":
            plt.figure()
            ax=plt.subplot(211)
            ax.set_xlim(xyrange)
            ax.set_ylim(xyrange)
            plt.imshow(abs(ft), **plot_args)
            plt.subplot(212)
            plt.imshow(ift)
    return(ift)

# also used for clustering?
def local_maximum(mx, r):
    tmp=ndimage.maximum_filter(mx, size=r)
    return(np.where(mx == tmp, mx, 0))

def mainFFT(ft, n=5, isSep=1, xylen=10, **plot_args):
    """Plot the top N FT frequencies separately and their corresponding waves, return the complex values as a list 

    """
    ft_sort=np.unravel_index(np.argsort(-abs(ft), axis=None),ft.shape)
    a, _ =ft.shape
    xyrange = [a//2-xylen, a//2+xylen]
    if isSep == 1:
        plt.figure(figsize=[5*(n+2),10])
        plt.subplot(2,2+n,1)
        plt.imshow(iFFT(ft))
        ax=plt.subplot(2,2+n,3+n)
        ax.set_xlim(xyrange)
        ax.set_ylim(xyrange)
        nft=np.zeros([64,64],dtype=complex)
        nft[ft_sort[0][1:2*n+1],ft_sort[1][1:2*n+1]]=ft[ft_sort[0][1:2*n+1],ft_sort[1][1:2*n+1]]
        plt.imshow(np.abs(nft), **plot_args)
        ax=plt.subplot(2,2+n,4+n)
        ax.set_xlim(xyrange)
        ax.set_ylim(xyrange)
        plt.imshow(local_maximum(np.abs(nft),3), cmap = 'PuOr', **plot_args)    
        for i in range(n):
            nft=np.zeros([64,64],dtype=complex)
            nft[ft_sort[0][2*i+1:2*(i+1)+1],ft_sort[1][2*i+1:2*(i+1)+1]]=ft[ft_sort[0][2*i+1:2*(i+1)+1],ft_sort[1][2*i+1:2*(i+1)+1]]
            #nft[ft_sort[0][i+1],ft_sort[1][i+1]]=ft[ft_sort[0][i+1],ft_sort[1][i+1]]
            #nft[ft_sort[0][i],ft_sort[1][i]]=ft[ft_sort[0][i],ft_sort[1][i]]
            ax=plt.subplot(2,2+n,n+i+5)
            ax.set_xlim(xyrange)
            ax.set_ylim(xyrange)
            plt.imshow(np.abs(nft), **plot_args)
            plt.subplot(2,2+n,i+2+1)
            plt.imshow(iFFT(nft), **plot_args)
    else:
        nft=np.zeros([64,64],dtype=complex)
        nft[ft_sort[0][1:n*2+1],ft_sort[1][1:n*2+1]]=ft[ft_sort[0][1:n*2+1],ft_sort[1][1:n*2+1]]
        plt.figure(figsize=[9,3])
        plt.subplot(1,3,1)
        plt.imshow(iFFT(ft), **plot_args)
        ax=plt.subplot(1,3,2)
        ax.set_xlim(xyrange)
        ax.set_ylim(xyrange)
        plt.imshow(np.abs(nft), cmap="PuOr", **plot_args)
        plt.subplot(1,3,3)
        plt.imshow(iFFT(nft), **plot_args)

    return([ft_sort[0][1:2*(n+1)+1],ft_sort[1][1:2*(n+1)+1]], ft[ft_sort[0][1:2*(n+1)+1],ft_sort[1][1:2*(n+1)+1]])


def unit_circle(n, rrange):
    A = np.arange(-n+0.5,n+0.5)**2
    dists = np.sqrt(A[:,None] + A)
    return ((dists-rrange[0]>0.5) & (rrange[1]-dists >= -0.5)).astype(int)


# The functions for testing differences among the 4 scenarios
def plot_sum_r(ft, r):
    freqs=[ np.sum(np.real(ft)*unit_circle(32,x)) for x in range(r)]
    plt.plot(freqs)

def plot_max_r(ft, r):
    freqs=[ np.max((np.real(ft)*unit_circle(32,x)).flatten()) for x in range(r)]
    plt.plot(freqs/freqs[1])


def plot_ratio_r(ft, r):
    ratios=[ np.max((np.real(ft)*unit_circle(32,x)).flatten())/(np.min((np.real(ft)*unit_circle(32,x)).flatten())+1) for x in range(r)]
    plt.plot(ratios/ratios[1])

def plot_max3_r(ft, r):
    freqs=[ np.sort((np.real(ft)*unit_circle(32,x)).flatten())[-3]/np.max((np.real(ft)*unit_circle(32,1)).flatten()) for x in range(r)]
    plt.plot(freqs/freqs[1])

# def plot_max3_r(ft, r):
#     ratios=[ np.max((np.real(ft)*unit_circle(32,x)).flatten())/(np.sort((np.real(ft)*unit_circle(32,x)).flatten())[-3]+1) for x in range(r)]
#     ratios[1]=0
#     plt.plot(ratios)

def plot_avg6_r(ft, r):
    ratios = [ np.mean(np.sort((np.real(ft)*unit_circle(32,x)).flatten())[-6:])/np.max((np.real(ft)*unit_circle(32,1)).flatten()) for x in range(r) ]
    plt.plot(ratios)
    return [np.sort((np.real(ft)*unit_circle(32,x)).flatten())[-6:] for x in range(r)]

## Simulation Utils

### PSF

In [ ]:
PSF=[[] for x in range(20)]
for size in range(20):
    sigma_y = sigma_x = 0.5
    k=1
    x = np.linspace(-k, k, size)
    y = np.linspace(-k, k, size)
    x, y = np.meshgrid(x, y)
    PSF[size] = 255 * np.exp(-(x**2/(2*sigma_x**2)+ y**2/(2*sigma_y**2)))
    # plt.figure()
    # plt.imshow(PSF[size])

### Simulate 1D, 2D, 2DC, Random points, 1Dgraph

In [ ]:
def genPic(matrix, locs, point=9, theta=0, rand=0, randz=0):
    sx,sy = PSF[point].shape
    mx,my = matrix.shape
    tmp_matrix=np.pad(matrix, (sx, sy), mode="constant", constant_values=0)
    for px, py in locs:
        if rand!=0:
            px+=np.random.randint(2*rand)-rand
            py+=np.random.randint(2*rand)-rand
        tpoint = point+np.random.randint(randz)
        px-=tpoint//2
        py-=tpoint//2
        if px < 0 or px+tpoint > mx or py < 0 or py+tpoint > my:
            continue
        tmp_matrix[px:px+tpoint, py:py+tpoint] += PSF[tpoint]
    if theta != 0:
        rotated = ndimage.rotate(tmp_matrix, theta)
        tmp,_ = rotated.shape
        return(rotated[tmp//2-mx//2:tmp//2+mx//2,tmp//2-my//2:tmp//2+my//2])
    else:
        return(tmp_matrix[sx:-sx,sy:-sy])

def sim1D(size=128, spacex=8, spacey=11, shift=0):
    points=[]
    for py in range(1, size, spacey):
        for px in range(1+py//int(spacey)*shift, size, spacex):
            points.append([px,py])
    return(np.array(points))

def sim2D(size=128, space=6):
    nx=ny=int(size/space)
    return np.array([[int((x*2-y%2)*space)+10, int(y*np.sqrt(3)*space)] for x in range(nx) for y in range(ny)])

def sim2DC(size=128, space=6):
    N_points=sim2D(size, space)
    N_dist = [[np.linalg.norm(i-j) for i in N_points] for j in N_points ]
    thre = space*2+1
    C_points = []
    n_vx = len(N_points)
    for i in range(n_vx):
        for j in range(i+1, n_vx):
            if N_dist[i][j] <= thre:
                C_points.append([(N_points[i][0]+N_points[j][0])//2, (N_points[i][1]+N_points[j][1])//2])
    return(np.array(C_points))

def simRand(size=128,space=6):
    return(np.random.randint(size,size=(int(size*size/space/space),2)))

# simulate continuous graph and test

def sim_d(d, c_point, len, dist, width, w_d=3):
    points=[]
    for x in range(1, len+1):
        point=[[c_point[0]+width*np.sin(d)*y, c_point[1]-width*np.cos(d)*y] for y in range(-w_d, w_d)]
        #print(x)
        #print(point)
        points.extend(point)
        c_point=[c_point[0]+dist*np.cos(d), c_point[1]+dist*np.sin(d)]
    return(c_point, points)


def sim_1dgraph(c_point=[50,50], d=np.pi/4, dist=11, width=8, **genPic_args):
    points=[]
    # for i in range(3):
    #     tmp_d=2*np.pi*np.random.rand()
    for tmp_d in [90-np.random.randint(18)*10 for x in range(10)]:
        tmp_d = tmp_d/180*np.pi
        for j in np.linspace(0,tmp_d, 10):
            c_d = d+j
            length = np.random.randint(5)
            c_point, c_points=sim_d(d=c_d, c_point=c_point, len=length, dist=dist, width=width)
            #print(c_d, c_point, len)
            points.extend(c_points)
        d = d+tmp_d
        if(np.max(np.abs(c_point)))> 1080 or c_point[0]<0 or c_point[1]<0:
            break
        c_point, c_points=sim_d(d=c_d, c_point=c_point, len=np.random.randint(5), dist=dist, width=width)
        points.extend(c_points)
        if(np.max(np.abs(c_point)))> 1080 or c_point[0]<0 or c_point[1]<0:
            break

    points=np.array(points).astype(int)
    simD = genPic(np.zeros([1280,1280]), points, **genPic_args)
    plt.figure(figsize=[10,10])
    plt.imshow(simD)
    return simD

## Classify Utils

In [ ]:
def maximum (A, B):
    BisBigger = A-B
    BisBigger = np.where(BisBigger < 0, 1, 0)
    return A - A*(BisBigger) + B*(BisBigger)

def maskMat(img, label, kernel_size=64, steps=16):
    if len(img.shape)==3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    xn, yn=img.shape
    x_l = [ x for x in range(0,xn-kernel_size+1, steps)]
    y_l = [ y for y in range(0,yn-kernel_size+1, steps)]
    matrix = np.zeros([xn, yn])
    for i in range(len(x_l)):
        for j in range(len(y_l)):
            if(label[i,j]):
                xi = x_l[i]
                yj = y_l[j]
                matrix[xi:xi+kernel_size,yj:yj+kernel_size] = img[xi:xi+kernel_size,yj:yj+kernel_size]
    return matrix

def maxMat(img, c, kernel_size=64, steps=16):
    if len(img.shape)==3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    xn, yn=img.shape
    x_l = [ x for x in range(0,xn-kernel_size+1, steps)]
    y_l = [ y for y in range(0,yn-kernel_size+1, steps)]
    matrix = np.zeros([xn, yn])
    for i in range(len(x_l)):
        for j in range(len(y_l)):
            xi = x_l[i]
            yj = y_l[j]
            matrix[xi:xi+kernel_size,yj:yj+kernel_size] = maximum(matrix[xi:xi+kernel_size,yj:yj+kernel_size], c[i,j])
    return matrix

def sumMat(img, c, kernel_size=64, steps=16, method="default"):
    if len(img.shape)==3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    xn, yn=img.shape
    x_l = [ x for x in range(0,xn-kernel_size+1, steps)]
    y_l = [ y for y in range(0,yn-kernel_size+1, steps)]
    print(len(x_l), len(y_l))
    matrix = np.zeros([xn, yn])
    kernel = np.ones([kernel_size,kernel_size])

    if method == "gaussian":
        g = cv2.getGaussianKernel(kernel_size, sigma=0)
        kernel = g*g.T
    if method == "flattop":
        g = cv2.getGaussianKernel(kernel_size, sigma=kernel_size/2)
        kernel = g*g.T
    for i in range(len(x_l)):
        for j in range(len(y_l)):
            if(c[i,j]>0):
                xi = x_l[i]
                yj = y_l[j]
                matrix[xi:xi+kernel_size,yj:yj+kernel_size] += c[i,j]*kernel
    return matrix

def fft_filter(img, rrange, max=6, plot=0, kernel_size=64, corr="Pearson", method="fft", **plot_args):
    ft=FFT(img)
    if method == "auto" or method=="auto2":
        img=scipy.signal.fftconvolve(img, np.flip(img), mode = 'same')
        ft=FFT(img.astype('float32'))
    if method == "conv":
        img=scipy.signal.fftconvolve(img, img, mode = 'same')
        ft=FFT(img.astype('float32'))
    if method == "rb_auto":
        img=scipy.signal.fftconvolve(img, np.flip(img), mode = 'same')
    if method == "bin":
        a=scipy.signal.fftconvolve(img, np.flip(img), mode = 'same')
        tmp = 255* a.astype(np.float64) / a.max()
        a = tmp.astype(np.uint8)
        local = 7
        img = cv2.adaptiveThreshold(a, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, local, 0)
        ft=FFT(img)
    if method == "bin1":
        a=scipy.signal.fftconvolve(img, np.flip(img), mode = 'same')
        tmp = 255* a.astype(np.float64) / a.max()
        a = tmp.astype(np.uint8)
        ret1, img = cv2.threshold(a, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
        #img = cv2.adaptiveThreshold(a, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, local, 0)
        ft=FFT(img)
    if len(img.shape)==3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    sn,_ = ft.shape
    ft_sort=np.unravel_index(np.argsort(-abs(ft), axis=None),ft.shape)
    nft=np.zeros([kernel_size,kernel_size],dtype=complex) 
    nft[ft_sort[0][1:2*(max+1)],ft_sort[1][1:2*(max+1)]]=ft[ft_sort[0][1:2*(max+1)],ft_sort[1][1:2*(max+1)]]
    tmp = np.abs(nft)*unit_circle(sn//2, rrange) 

    if tmp.argmax()==0:   # that there's no pattern at all
        return([-1, -1, -1])     
    dx, dy = np.unravel_index(tmp.argmax(), tmp.shape)
    nft=np.zeros([kernel_size,kernel_size],dtype=complex) 
    nft[dx, dy]=ft[dx, dy]
    nft[kernel_size-dx, kernel_size-dy]=ft[kernel_size-dx, kernel_size-dy]
    if plot=="v":
        plt.figure()
        plt.subplot(311)
        plt.imshow(img,**plot_args)
        plt.subplot(312)
        plt.imshow(np.abs(nft))
        plt.subplot(313)
        plt.imshow(iFFT(nft))    
    elif plot=="h":
        plt.figure()
        plt.subplot(131)
        plt.imshow(img, **plot_args)
        plt.subplot(132)
        plt.imshow(np.abs(nft))
        plt.subplot(133)
        plt.imshow(iFFT(nft))
    x = img.flatten()
    y = iFFT(nft).flatten()

    if method == "auto2":
        y = iFFT(nft)
        y = scipy.signal.fftconvolve(y, np.flip(y), mode = 'same')
        y = y.flatten()
    # all of these three lie in -1 to 1
    if corr == "Pearson":
        conf = scipy.stats.pearsonr(x, y)[0]
    elif corr == "Spearman":
        conf = scipy.stats.spearmanr(x, y).statistic
    elif corr == "Kendall":
        conf = scipy.stats.kendalltau(x, y).statistic
    degree = np.arctan((dy-sn//2)/(dx-sn//2+0.001))
    if degree <= 0:
        degree +=np.pi
    dist = kernel_size/np.sqrt((dy-sn//2)**2+(dx-sn//2)**2)
    degree = degree/np.pi*180
    return([conf, degree, dist])


def FFTfilter(img, rrange, max=6, kernel_size=64, threshold=0.1, sparse=500, steps=16, corr="Pearson", method="fft", type=0, plot=None, save_thre=None, save_name="save", save_dir="/Users/suestring/Desktop/Lab/Code/Hongyun/Test/", **plot_args):
    if len(img.shape)==3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    xn,yn = img.shape
    # if plot == "mask":
    #     mask = np.zeros([xn, yn])
    x_l = [ x for x in range(0,xn-kernel_size+1, steps)]
    y_l = [ y for y in range(0,yn-kernel_size+1, steps)]
    m_conf = np.zeros([len(x_l), len(y_l)])
    m_degree = np.zeros([len(x_l), len(y_l)])
    m_dist = np.zeros([len(x_l), len(y_l)])
    if save_thre is not None:
        pdf = matplotlib.backends.backend_pdf.PdfPages(save_dir+save_name+"_thre"+str(save_thre)+".pdf")
        
    for i in range(len(x_l)):
        for j in range(len(y_l)):
            xi = x_l[i]
            yj = y_l[j]
            if np.sum(img[xi:xi+kernel_size,yj:yj+kernel_size]) <= sparse:
                m_conf[i,j], m_degree[i,j], m_dist[i,j] = [-1, -1, 0]
                continue
            t_conf, t_degree, t_dist = fft_filter(img[xi:xi+kernel_size,yj:yj+kernel_size], rrange, max=max, kernel_size=kernel_size, corr=corr, method=method, type=type)
            if t_conf < threshold:
                m_conf[i,j], m_degree[i,j], m_dist[i,j] = [-1, -1, 0]
            else:
                m_conf[i,j], m_degree[i,j], m_dist[i,j] = t_conf, t_degree, t_dist
                # if plot == "mask":
                #     mask[xi:xi+64,yj:yj+64]=img[xi:xi+64,yj:yj+64]
            if save_thre is not None:
                if t_conf > save_thre:
                    fig = plt.figure()
                    timg=img[xi:xi+kernel_size,yj:yj+kernel_size]
                    #fft_filter(img[xi:xi+kernel_size,yj:yj+kernel_size], rrange, max=max, kernel_size=kernel_size, corr=corr, method=method, plot="h")
                    #a=scipy.signal.fftconvolve(timg, np.flip(timg), mode = 'same')
                    #tmp = 255* a.astype(np.float64) / a.max()
                    #a = tmp.astype(np.uint8)
                    #local = 7
                    #timg = cv2.adaptiveThreshold(a, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, local, 0)
                    plt.imshow(timg)
                    plt.title("x"+str(xi)+"_y"+str(yj)+"_conf"+str(t_conf)+"_orient"+str(int(t_degree)))
                    pdf.savefig(fig)
    # if plot == "mask":
    #     plt.figure()
    #     plt.imshow(mask)
    if plot is not None:
        plt.figure(**plot_args)
        plt.subplot(131)
        plt.imshow(m_conf,cmap="PuOr", vmin=-1, vmax=1)
        plt.subplot(132)
        plt.imshow(m_degree,cmap="PuOr", vmin=0, vmax=180)
        plt.subplot(133)
        plt.imshow(m_dist,cmap="PuOr")
    if save_thre is not None:
        pdf.close()
    return m_conf, m_degree, m_dist

def plotDegree(mat_d):
    dx, dy=mat_d.shape
    _, axes = plt.subplots(dx, dy,sharex=True, sharey=True)
    for i in range(dx):
        for j in range(dy):
            axes[i][j].plot([x for x in range(10)],[x*np.tan(mat_d[i][j]+np.pi/2) for x in range(10)])

In [ ]:
g = np.array([ [np.power(x,0.7)] for x in range(32)])
g=np.concatenate((g,g[::-1]))
kernel = g*g.T
plt.imshow(kernel,vmin=0)

In [ ]:
# flat top
g=cv2.getGaussianKernel(64, sigma=32)
kernel = g*g.T
#kernel[kernel>0.4]=0.4
plt.imshow(kernel)

In [ ]:
kernel_size=64
g = cv2.getGaussianKernel(kernel_size, sigma=0)
kernel = g*g.T
#kernel = np.ones([64,64])
plt.imshow(kernel,vmin=0)

# Test(Done) TODO:sort

## Test 1D

### Randomness increase

In [ ]:

theta = 30
points=sim1D()

#for tsize in [1,4,9]:
for rand in [0,1,2,3,4]:
    sim_1D = genPic(np.zeros([128,128]), points, 9, theta=theta, rand=0, randz=5)
    sim_1D = genPic(np.zeros([128,128]), points, 9, theta=theta, rand=0, randz=5)
    #ft=FFTplot(sim_1D[32:96,32:96], plot=0)
    #_=abs_mainFFT(ft,5)
    #_=mainFFT(ft,n=4, xyrange=[22,42])
    print(fft_filter(sim_1D[32:96,32:96], [5,8], max=5, plot="h"))

In [ ]:
test=cv2.imread("/Users/suestring/Desktop/Lab/Code/Hongyun/data/test/2D/adducin_ac2_72_131_64.png")
ft=FFTplot(test)
ft[32,32]=0
ft_sort=np.unravel_index(np.argsort(-np.real(ft), axis=None),ft.shape)

for i in range(5):
    nft=np.zeros([64,64],dtype=complex)
    nft[ft_sort[0][2*i:2*(i+1)],ft_sort[1][2*i:2*(i+1)]]=ft[ft_sort[0][2*i:2*(i+1)],ft_sort[1][2*i:2*(i+1)]]
    # plt.figure()
    # plt.subplot(1,2,1)
    # plt.imshow(np.real(nft))
    # plt.subplot(1,2,2)
    iFFTplot(nft)

plt.figure()
nft=np.zeros([64,64],dtype=complex)
nft[ft_sort[0][:2*(i+1)],ft_sort[1][:2*(i+1)]]=ft[ft_sort[0][:2*(i+1)],ft_sort[1][:2*(i+1)]]
#nft[:,:]=ft[:,:]

ift=iFFTplot(nft)

## Test data

In [ ]:
test_1d=glob.glob("/Users/suestring/Desktop/Lab/Code/Hongyun/data/test/1D/*.png")
test_2d=glob.glob("/Users/suestring/Desktop/Lab/Code/Hongyun/data/test/2D/*.png")
test_sparse=glob.glob("/Users/suestring/Desktop/Lab/Code/Hongyun/data/test/sparse/*.png")
test_axon = cv2.imread("/Users/suestring/Desktop/Lab/Code/Hongyun/data/rendered/axon_x20.png")

In [ ]:
plt.figure(figsize=[5*len(test_1d),5])
for i in range(len(test_1d)):
    plt.subplot(1, len(test_1d),i+1)
    test=cv2.imread(test_1d[i])
    test =cv2.cvtColor(test, cv2.COLOR_BGR2GRAY)
    plt.imshow(test)

plt.figure(figsize=[5*len(test_2d),5])
for i in range(len(test_2d)):
    plt.subplot(1, len(test_2d),i+1)
    test=cv2.imread(test_2d[i])
    test =cv2.cvtColor(test, cv2.COLOR_BGR2GRAY)
    plt.imshow(test)

plt.figure(figsize=[5*len(test_sparse),5])
for i in range(len(test_sparse)):
    plt.subplot(1, len(test_sparse),i+1)
    test=cv2.imread(test_sparse[i])
    test =cv2.cvtColor(test, cv2.COLOR_BGR2GRAY)
    plt.imshow(test)

In [ ]:
test_axon1 = test_axon[875:875+64, 380:380+64]
ft=FFTplot(test_axon1,plot=0)
_=mainFFT(ft,3)

In [ ]:
for i in test_1d:
    test=cv2.imread(i)
    ft=FFTplot(test,plot=0)
    _=mainFFT(ft,5)

In [ ]:
for i in test_2d:
    test=cv2.imread(i)
    ft=FFTplot(test,plot=0)
    mainFFT(ft,12,isSep=0)

In [ ]:
for i in test_sparse:
    test=cv2.imread(i)
    ft=FFTplot(test,plot=0)
    _=mainFFT(ft,3)

### Test Axon

In [ ]:
test = cv2.imread("../data/rendered/axon_x20.png")
#test1=test[550:614,500:564]
test1 = test[500:650, 500:600]
plt.figure()
plt.imshow(test1)
#ft=FFTplot(test1, plot=0)
#_=mainFFT(ft,6)
#_=abs_mainFFT(ft,6)
#print(fft_filter(test1, [2,4], plot="h"))
_=FFTfilter(test1, [2,4], kernel_size=64, threshold=0.1, sparse=0, steps=8)
#test2=test[40:104,150:214]
test2=test[0:104,0:214]
plt.figure()
plt.imshow(test2)
#ft=FFTplot(test2, plot=0)
#_=mainFFT(ft,6)
#print(fft_filter(test2, [2,4], plot="h"))
_=FFTfilter(test2, [2,4], kernel_size=64, threshold=0.1, sparse=0, steps=8, corr="Pearson")

In [ ]:
test = cv2.imread("../data/rendered/axon_x20.png")
#test1=test[550:614,500:564]
test1 = test[500:650, 500:600]
plt.figure()
plt.imshow(test1)
#ft=FFTplot(test1, plot=0)
#_=mainFFT(ft,6)
#_=abs_mainFFT(ft,6)
#print(fft_filter(test1, [2,4], plot="h"))
_=FFTfilter(test1, [2,4], kernel_size=64, threshold=0.1, sparse=0, steps=8)
#test2=test[40:104,150:214]
test2=test[0:104,0:214]
plt.figure()
plt.imshow(test2)
#ft=FFTplot(test2, plot=0)
#_=mainFFT(ft,6)
#print(fft_filter(test2, [2,4], plot="h"))
_=FFTfilter(test2, [2,4], kernel_size=64, threshold=0.1, sparse=0, steps=8, corr="Pearson")

In [ ]:
test = cv2.imread("../data/rendered/axon_x20.png")
#test1=test[550:614,500:564]
test1 = test[500:650, 500:600]
plt.figure()
plt.imshow(test1)
#ft=FFTplot(test1, plot=0)
#_=mainFFT(ft,6)
#_=abs_mainFFT(ft,6)
#print(fft_filter(test1, [2,4], plot="h"))
_=FFTfilter(test1, [2,4], kernel_size=64, threshold=0.1, sparse=0, steps=8)
#test2=test[40:104,150:214]
test2=test[0:104,0:214]
plt.figure()
plt.imshow(test2)
#ft=FFTplot(test2, plot=0)
#_=mainFFT(ft,6)
#print(fft_filter(test2, [2,4], plot="h"))
_=FFTfilter(test2, [2,4], kernel_size=64, threshold=0.1, sparse=0, steps=8, corr="Pearson")

In [ ]:
test = cv2.imread("../data/rendered/axon_x20.png")
#test1=test[550:614,500:564]
test1 = test[500:650, 500:600]
plt.figure()
plt.imshow(test1)
#ft=FFTplot(test1, plot=0)
#_=mainFFT(ft,6)
#_=abs_mainFFT(ft,6)
#print(fft_filter(test1, [2,4], plot="h"))
_=FFTfilter(test1, [2,4], kernel_size=64, threshold=0.1, sparse=0, steps=8)
#test2=test[40:104,150:214]
test2=test[0:104,0:214]
plt.figure()
plt.imshow(test2)
#ft=FFTplot(test2, plot=0)
#_=mainFFT(ft,6)
#print(fft_filter(test2, [2,4], plot="h"))
_=FFTfilter(test2, [2,4], kernel_size=64, threshold=0.1, sparse=0, steps=8, corr="Pearson")

## Simulated data

In [ ]:
def simPattern(i, space):
    if i==0:
        return(sim1D())
    elif i==1:
        return(sim2D(space=space))
    elif i==2:
        return(sim2DC(space=space))
    else:
        return(simRand(space=space))

plt.figure(figsize=[40,20])

for i in range(4):
    for j in range(8):
        theta = np.random.randint(0,180)
        sim_tmp=genPic(np.zeros([128,128]), simPattern(i, space=j+5), 10, theta, rand=2, randz=5)
        plt.subplot(4, 8, i*8+j+1)
        plt.imshow(sim_tmp[32:96, 32:96])



### 1D pattern

In [ ]:
p_size = 16
thre = 190/p_size
nx = ny = 10
space=8
for shift in [0,3,5]:
    points=[]
    for py in range(1, 128, int(thre)):
        for px in range(1+py//int(thre)*shift, 128, space):
            points.append([px,py])
    
    for theta in [0,17,129]:
        sim_1D = genPic(np.zeros([128,128]), points, 9, theta, rand=2, randz=6)
        ft=FFTplot(sim_1D[32:96,32:96], plot=0)
        _=mainFFT(ft,5)
        plt.figure()
        #plot_max3_r(ft,20)
        #inv_mainFFT(ft,12)

In [ ]:
nfft = 10
psize = 9
points=sim1D()
for nRand in [0, 10, 50, 100]:
    sim_1D = genPic(np.zeros([128,128]), np.vstack((points,np.random.randint(32, 96,[nRand,2]))), psize, theta, rand=0, randz=2)
    #ft=FFTplot(sim_1D[32:96,32:96], plot=0)
    #plt.figure()
    #_=mainFFT(ft,nfft,isSep=0,xylen=10)
    print(fft_filter(sim_1D[32:96,32:96], [5,7], plot="h", vmin=-10, vmax=800))

In [ ]:
sim_1D = genPic(np.zeros([128,128]), points, 9, theta, rand=2, randz=6)
ft=FFTplot(sim_1D[32:96,32:96], plot=0)
for nfft in range(1,10):
    plt.figure()
    _=mainFFT(ft,nfft,isSep=0,xylen=10)

### 2D pattern

#### Junction(Spectrin-N, Adducin)

In [ ]:
## 2D pattern
nx = 12
ny = 12
sim_2D=np.zeros([64,64])
space = int(100/p_size)
sim_points=np.array([[int((x*2-y%2)*space)+10, int(y*np.sqrt(3)*space)] for x in range(nx) for y in range(ny)])

theta=120
#for theta in range(1,180,36):
sim_2D = genPic(np.zeros([128,128]), sim2D(space=7), 9, theta, rand=3, randz=4)
for nfft in range(1,7):
    ft=FFTplot(sim_2D[32:96,32:96], plot=0)
    #_=mainFFT(ft,6)
    plt.figure()
    # plot_max_r(ft,20)
    _=mainFFT(ft,nfft,isSep=0,xylen=10)

#### Spectrin-C

In [ ]:
#space = int(100/p_size)
space = 12
sim_points=np.array([[int((x*2-y%2)*space)+10, int(y*np.sqrt(3)*space)] for x in range(nx) for y in range(ny)])
sim_dist = [[np.linalg.norm(i-j) for i in sim_points] for j in sim_points ]
thre = 2*space+1

c_points = []
n_vx = len(sim_points)
for i in range(n_vx):
    for j in range(i+1, n_vx):
        if sim_dist[i][j] <= thre:
            c_points.append([(sim_points[i][0]+sim_points[j][0])//2, (sim_points[i][1]+sim_points[j][1])//2])

nfft = 12
#for theta in range(1,180,30):
theta = 30    
sim_2D_C = genPic(np.zeros([128,128]), sim2DC(space=9), 9, theta=theta, rand=0, randz=1)
for nfft in [12]:#1,3,6,9,10,12]:
    ft=FFTplot(sim_2D_C[32:96,32:96], plot=0)
    #_=mainFFT(ft,10)
    plt.figure()
    #plot_max_r(ft,20)
    ind, comp=mainFFT(ft,nfft,isSep=0,xylen=10)

### Random pattern

In [ ]:
for i in range(7):
    sim_2D_C = genPic(np.zeros([128,128]), simRand(space=12), 9, rand=2, randz=6)
    ft=FFTplot(sim_2D_C[32:96,32:96])
    #_=mainFFT(ft,6)
    plt.figure()
    #plot_max_r(ft,20)

# Playground

In [ ]:
img=cv2.imread("../data/rendered/test/2D/adducin_ac2_72_131_64.png")
test =cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
c,o,d =FFTfilter(test, [5,7], kernel_size=64)

In [ ]:
rendered = cv2.imread("../data/rendered/Axon_x20.png")
test =cv2.cvtColor(rendered, cv2.COLOR_BGR2GRAY)
#test1=test[400:600,500:600]
plt.imshow(test)
c,o,d =FFTfilter(test, [2,4],kernel_size=64, threshold=0, sparse=500, steps=16)

In [ ]:
simD=sim_1dgraph(rand=1,randz=2)

In [ ]:
plt.figure()
space=[5,7]
max=8
steps=8
corr="Pearson"
for method in ["fft","auto","bin"]:
    c,o,d =FFTfilter(simD, space, kernel_size=64, max=max, threshold=0, sparse=10000, steps=steps, corr=corr,method=method,figsize=[15,5], plot=1)

In [ ]:
a=[]
for x in range(10):
    for y in range(1):
        a.append(fft_filter(simD[x:x+64,y:y+64],space, plot="h", method="bin1")[0])

In [ ]:
plt.plot(a)

In [ ]:
corr="Pearson"
theta=0


space=[5,7]
for i in [10, 20, 40]:
    simDnR=genPic(simD, simRand(1280,space=i), 9, theta=theta, rand=3, randz=8)
    plt.figure()
    plt.imshow(simDnR)
    for method in ["fft","auto","bin","bin1"]:
        c,o,d =FFTfilter(simDnR, space, kernel_size=64, threshold=0, sparse=1000, steps=steps, corr=corr, method=method,figsize=[15,5])

In [ ]:

for i in test_1d:
    test=cv2.imread(i)
    test_conv=scipy.signal.fftconvolve(test, np.flip(test), mode = 'same')
    ft=FFTplot(test,plot=0)
    _=mainFFT(ft,5)
    ft=FFTplot(test_conv.astype('float32'),plot=0)
    _=mainFFT(ft,5)

In [ ]:
rendered = cv2.imread("/Users/suestring/Desktop/Lab/Code/Hongyun/data/rendered/A1.png")
p_size = 16
space = int(190/p_size)
test =cv2.cvtColor(rendered, cv2.COLOR_BGR2GRAY)
test1 = test[1500:2400,1500:2400]
#test2 = test[1800:1900,1700:1800]
#test1 = test[1800:2000,1700:1900]
plt.imshow(test1)
space=[5,8]
steps=8
plt.figure()
c,o,d =FFTfilter(test1, space, kernel_size=64, threshold=0, sparse=10000, steps=steps, corr=corr)
#c,o,d =FFTfilter(test1, space, kernel_size=64, threshold=0, sparse=10000, steps=steps, corr=corr, method="auto")

In [ ]:
def findParameter(img, xy, kernel_size):
    #plt.imshow(img[xy[0]:xy[0]+kernel_size, xy[1]:xy[1]+kernel_size])
    test = img[xy[0]:xy[0]+kernel_size, xy[1]:xy[1]+kernel_size]
    ft = FFTplot(test, plot=0)
    _ = mainFFT(ft, n=5)
    print(np.sum(test))

In [ ]:
# 7.2nm per pixel, thus 190/7.2
#rendered = cv2.imread("/Users/suestring/Desktop/Lab/Code/Hongyun/data/rendered/2color_spectrin_Cterm.jpg")
rendered = cv2.imread("/Users/suestring/Desktop/Lab/Code/Hongyun/data/rendered/A1.png")

test =cv2.cvtColor(rendered, cv2.COLOR_BGR2GRAY)

plt.imshow(test)
test1=test
# test1=test[1200:2400, 1600:2800]

space=[5,8]
corr="Pearson"
steps=8
c=[[] for i in range(3)]
methods = ["bin"]#fft", "auto", "bin"]

pixel_size=16
sx, sy=test1.shape
l_ratios=[ (x*180/64/pixel_size) for x in range(2,10)]
#xylist=[[430, 290],[600,400],[570,390]]
spectrin_range=np.array([200, 140])  
kernel_size = 64
rrange=(kernel_size*pixel_size/spectrin_range).astype(int)
rrange[1]+=1

ratio=l_ratios[1]#,1,1.5,2]:
t_test = cv2.resize(test1, (int(sy/ratio), int(sx/ratio)))

t_pixel = pixel_size*ratio
rrange=(kernel_size*t_pixel/spectrin_range).astype(int)
rrange[1]+=1
method=methods[0]
c[i],o,d =FFTfilter(t_test, space, kernel_size=64, threshold=0.2, sparse=100000, steps=8, corr=corr, method=method, save_thre=0.5,save_name="stripe3_raw")
plt.figure(figsize=[10,10])
a=sumMat(t_test,c[i],steps=8, method="flattop")
plt.imshow(cv2.merge([t_test,(255*a/np.max(a,axis=None)).astype(np.uint8),t_test]))


In [ ]:
l_ratios

In [ ]:
n_size=test1.shape
nc=np.array(Image.fromarray(c).resize(n_size))
plt.imshow(test1)
plt.imshow(nc, vmin=0.2, vmax=0.5, alpha=0.5,cmap="magma")

In [ ]:
plt.figure(figsize=[10,5])
plt.subplot(121)
plt.imshow(test1)

plt.subplot(122)
plt.imshow(test1)
plt.imshow(nc, vmin=0.25, vmax=0.5, alpha=0.3,cmap="hsv")


In [ ]:
plt.imshow(cv2.merge([test1,maskMat(test1,c>0.36,steps=8).astype(np.uint8),test1]))

In [ ]:
plt.imshow(sumMat(test1,c,steps=8))

In [ ]:
for i in range(3):
    plt.figure(figsize=[10,10])
    a=sumMat(test1,c[i],steps=8, method="flattop")
    plt.imshow(cv2.merge([test1,(255*a/np.max(a,axis=None)).astype(np.uint8),test1]))

In [ ]:
a=sumMat(test1,c,steps=8, method="gaussian")

In [ ]:
plt.imshow(255*(a/np.max(a,axis=None)))

In [ ]:
if(-1):
    print(3)

In [ ]:
test1.shape

In [ ]:
oo=o.copy()
oo[np.where(c<0.3)]=0
plt.imshow(oo,vmin=0,vmax=180,cmap="PuOr",norm=None)

In [ ]:
plt.hist(7.2*np.sort(np.delete(d.flatten(), np.where(d.flatten()==0)),axis=None))

In [ ]:
np.delete(d, np.where(d==0))

In [ ]:
a=np.array([[1,2],[3,4]]).flatten()
np.delete(a, np.where(a==2))

In [ ]:
rendered = cv2.imread("/Users/suestring/Desktop/Lab/Code/Hongyun/data/rendered/NA1.png")
test =cv2.cvtColor(rendered, cv2.COLOR_BGR2GRAY)
plt.imshow(test[1000:1600,900:1500])
space=[5,8]

test1=test[1000:1600,900:1500]

space=[5,8]
corr="Pearson"
steps=8
c=[[] for i in range(3)]
methods = ["fft", "auto", "bin"]
for i in range(3):
    method=methods[i]
    c[i],o,d =FFTfilter(test1, space, kernel_size=64, threshold=0.2, sparse=100000, steps=8, corr=corr, method=method)#, save_thre=0.4)
    plt.figure(figsize=[10,10])
    a=sumMat(test1,c[i],steps=8, method="gaussian")
    plt.imshow(cv2.merge([test1,(255*a/np.max(a,axis=None)).astype(np.uint8),test1]))
# plt.figure()
# plt.subplot(121)
# plt.imshow(c,cmap="PuOr")
# plt.subplot(122)
# plt.imshow(d,cmap="rainbow")

In [ ]:
test2=test[1000:1600,900:1500]
n_size=test2.shape
nc=np.array(Image.fromarray(c1).resize(n_size))
plt.figure(figsize=[10,5])
plt.subplot(121)
plt.imshow(test2)

plt.subplot(122)
plt.imshow(test2)
plt.imshow(nc, vmin=0.25, vmax=0.5, alpha=0.3,cmap="hsv")

In [ ]:
#rendered = cv2.imread("/Users/suestring/Desktop/Lab/Code/Hongyun/data/rendered/NA5.png")
rendered = cv2.imread("../data/rendered/soma/1226_5.png")
test =cv2.cvtColor(rendered, cv2.COLOR_BGR2GRAY)
plt.imshow(test)#[1000:1600,900:1500])
#space=[3,6]
space=[5,7]
test1=test[700:1500,1000:1900] #1226_5
#test1=test[600:1600,600:1600] #NA5
#test1=test[0:1500,0:1500]  # NA2
corr="Pearson"
steps=8
c=[[] for i in range(3)]
methods = ["bin"]
for i in range(1):
    method=methods[i]
    c[i],o,d =FFTfilter(test1, space, kernel_size=32, threshold=0.3, sparse=100000, max=10, steps=8, corr=corr, method=method)#, save_thre=0.4)
    plt.figure(figsize=[10,10])
    a=sumMat(test1,c[i],kernel_size=32, steps=8, method="gaussian")
    plt.imshow(cv2.merge([test1,(255*a/np.max(a,axis=None)).astype(np.uint8),test1]))

In [ ]:
plt.figure()
plt.imshow(cv2.merge([test,maskMat(test,c1>0.32,steps=8).astype(np.uint8),test]))

In [ ]:
rendered = cv2.imread("/Users/suestring/Desktop/Lab/Code/Hongyun/data/rendered.png")
test =cv2.cvtColor(rendered, cv2.COLOR_BGR2GRAY)
plt.imshow(test)#[1000:1600,900:1500])
space=[2,6]

for size in [32, 48, 64]:
    for method in ["fft", "auto", "rb_auto"]:
        c1,o1,d1 =FFTfilter(test, space, kernel_size=size, threshold=0, sparse=10000, steps=8, method=method)
        a=sumMat(test,c1,steps=8, method="gaussian")
        plt.figure()
        plt.imshow(cv2.merge([test,(255*(a/np.max(a,axis=None))).astype(np.uint8),test]))

In [ ]:
plt.imshow(sumMat(test,c1,steps=8,method="flattop"))

In [ ]:
plt.imshow(c1)

In [ ]:
rendered = cv2.imread("/Users/suestring/Desktop/Lab/Code/Hongyun/data/rendered/A1.png")
test =cv2.cvtColor(rendered, cv2.COLOR_BGR2GRAY)
test1=test[1300:2300,1400:2200]

space=[5,8]

plt.figure()
plt.imshow(test1)
xylist=[[430, 290],[600,400],[570,390]]
for x, y in xylist:
    plt.gca().add_patch(Rectangle((y,x),64,64,linewidth=1,edgecolor='r',facecolor='none'))

for x, y in xylist:
    test2=test1[x:x+64, y:y+64]
    plt.figure()
    plt.imshow(test2)
    space=[5,8]
    corr="Pearson"
    print(fft_filter(test2, space, plot="h"))


c1,o1,d1 =FFTfilter(test1, space, kernel_size=64, threshold=0, sparse=200000, steps=8)


sx, sy=test1.shape
nc=np.array(Image.fromarray(c1).resize([sy, sx]))
plt.figure(figsize=[10,5])
plt.subplot(121)
plt.imshow(test1)

plt.subplot(122)
plt.imshow(test1)
plt.imshow(nc, vmin=0.1, vmax=0.7, alpha=0.3,cmap="hsv")

#### Same Kernel size, different resolution

In [ ]:
rendered = cv2.imread("/Users/suestring/Desktop/Lab/Code/Hongyun/data/rendered/NA2.png")
test =cv2.cvtColor(rendered, cv2.COLOR_BGR2GRAY)
test1=test[1:1500,1:1500]

#Default unit: nm
pixel_size=16
sx, sy=test1.shape
l_ratios=[ (x*180/64/pixel_size) for x in [4,5]]
spectrin_range=np.array([200, 140])  
kernel_size = 64
rrange=(kernel_size*pixel_size/spectrin_range).astype(int)
rrange[1]+=1

for ratio in l_ratios:
    t_test = cv2.resize(test1, (int(sy/ratio), int(sx/ratio)))
    t_pixel = pixel_size*ratio
    rrange=(kernel_size*t_pixel/spectrin_range).astype(int)
    rrange[1]+=1
    corr="Pearson"
    c, o, d = FFTfilter(t_test, rrange, kernel_size=64, threshold=0.3, sparse=100000, steps=8, plot=0, method="bin")
    plt.figure()
    a=sumMat(t_test,c,steps=8, method="gaussian")
    plt.imshow(cv2.merge([t_test,(255*a/np.max(a,axis=None)).astype(np.uint8),t_test]))


In [ ]:
l_ratios=[ (x*180/64/pixel_size) for x in range(3,10)]

In [ ]:
import matplotlib.backends.backend_pdf


#### Same Resolution, different kernel size

In [ ]:
rendered = cv2.imread("/Users/suestring/Desktop/Lab/Code/Hongyun/data/rendered/A1.png")
test =cv2.cvtColor(rendered, cv2.COLOR_BGR2GRAY)
test1=test[1300:2300,1400:2200]

#Default unit: nm
pixel_size=16 
sx, sy=test1.shape


xylist=[[430, 290],[600,400],[570,390]]
spectrin_range=np.array([200, 140])  
kernel_size = 64
rrange=(kernel_size*pixel_size/spectrin_range).astype(int)
rrange[1]+=1

for kernel_size in [16,32,48,64]:
    plt.figure()
    plt.imshow(test1)
    rrange=(kernel_size*pixel_size/spectrin_range).astype(int)
    rrange[1]+=1
    print(rrange)
    for x, y in xylist:
        plt.gca().add_patch(Rectangle((y,x),kernel_size,kernel_size,linewidth=1,edgecolor='r',facecolor='none'))

    for x, y in xylist:
        test2=test1[x:x+kernel_size, y:y+kernel_size]
        plt.figure()
        plt.imshow(test2)
        corr="Pearson"
        print(fft_filter(test2, rrange, kernel_size=kernel_size,plot="h"))
        c1,o1,d1 =FFTfilter(test1[x:x+64, y:y+64], rrange, kernel_size=kernel_size, threshold=0, sparse=1000, steps=8)
        # tx, ty=test1.shape
        # nc=np.array(Image.fromarray(c1).resize([ty, tx]))
        # plt.figure(figsize=[10,5])
        # plt.subplot(121)
        # plt.imshow(test1)

        # plt.subplot(122)
        # plt.imshow(test1)
        # plt.imshow(nc, alpha=0.3,cmap="hsv")

In [ ]:
rendered = cv2.imread("../data/rendered/soma/1226_5.png")
test =cv2.cvtColor(rendered, cv2.COLOR_BGR2GRAY)
test1=test[600:1500,1000:1800]

space=[5,8]

# plt.figure()
# plt.imshow(test1)
# xylist=[[430, 290],[660,320],[570,390]]
# for x, y in xylist:
#     plt.gca().add_patch(Rectangle((y,x),64,64,linewidth=1,edgecolor='r',facecolor='none'))

# for x, y in xylist:
#     test2=test1[x:x+64, y:y+64]
#     plt.figure()
#     plt.imshow(test2)
#     space=[5,8]
#     corr="Pearson"
#     print(fft_filter(test2, space, plot="h"))


c1,o1,d1 =FFTfilter(test1, space, kernel_size=64, method="bin", threshold=0, sparse=10000, steps=16)

# sx, sy=test1.shape
# nc=np.array(Image.fromarray(c1).resize([sy, sx]))
# plt.figure(figsize=[10,5])
# plt.subplot(121)
# plt.imshow(test1)

# plt.subplot(122)
# plt.imshow(test1)
# plt.imshow(nc, vmin=0, vmax=1, alpha=0.3,cmap="rainbow")
# nnc=nc.copy()
# nnc[nnc<0]=0
# plt.figure()
# plt.imshow(nnc, cmap="rainbow")
plt.imshow(test1)
a=sumMat(test1,c1,steps=16,method="flattop")
plt.imshow(a, vmin=0.0002, vmax=0.001, alpha=0.3,cmap="rainbow")

In [ ]:

plt.imshow(test1)
a=sumMat(test1,c1,steps=16,method="flattop")
#a=sumMat(test1,c1,steps=16,method="gaussian")

#a=sumMat(test1,c1,steps=16)

thre = 4
#plt.imshow(a, vmin=thre, vmax=thre+0.1, alpha=0.3,cmap="rainbow")
plt.imshow(a, vmin=np.max(a)/2-0.0001, vmax=np.max(a), alpha=0.3,cmap="rainbow")

In [ ]:
np.max(a)

In [ ]:
np.max(a)

In [ ]:
rendered = cv2.imread("../data/rendered/soma/1226_5.png")
plt.imshow(rendered)

In [ ]:
rendered = cv2.imread("../data/rendered/Tcell/A1.png")
test =cv2.cvtColor(rendered, cv2.COLOR_BGR2GRAY)
test1=test[1300:2300,1400:2200]

space=[5,8]

plt.figure()
plt.imshow(test1)
xylist=[[430, 290],[660,320],[570,390]]
for x, y in xylist:
    plt.gca().add_patch(Rectangle((y,x),64,64,linewidth=1,edgecolor='r',facecolor='none'))

for x, y in xylist:
    test2=test1[x:x+64, y:y+64]
    plt.figure()
    plt.imshow(test2)
    space=[5,8]
    corr="Pearson"
    print(fft_filter(test2, space, plot="h"))


c1,o1,d1 =FFTfilter(test1, space, kernel_size=64, threshold=0, sparse=200000, steps=8)


sx, sy=test1.shape
nc=np.array(Image.fromarray(c1).resize([sy, sx]))
plt.figure(figsize=[10,5])
plt.subplot(121)
plt.imshow(test1)

plt.subplot(122)
plt.imshow(test1)
plt.imshow(nc, vmin=0.1, vmax=0.7, alpha=0.3,cmap="rainbow")
nnc=nc.copy()
nnc[nnc<0]=0
plt.imshow(nnc, cmap="rainbow")

In [ ]:
def fft_filter(img, rrange, max=6, plot=0, kernel_size=64, corr="Pearson", method="fft", type=0, **plot_args):
    ft=FFT(img)
    if method == "auto":
        img=scipy.signal.fftconvolve(img, np.flip(img), mode = 'same')
        ft=FFT(img.astype('float32'))
    elif method == "conv":
        img=scipy.signal.fftconvolve(img, img, mode = 'same')
        ft=FFT(img.astype('float32'))
    elif method == "rb_auto":
        img=scipy.signal.fftconvolve(img, np.flip(img), mode = 'same')
    elif method == "bin":
        a=scipy.signal.fftconvolve(img, np.flip(img), mode = 'same')
        tmp = 255* a.astype(np.float64) / a.max()
        a = tmp.astype(np.uint8)
        if type == 0:
            local = 7
            img = cv2.adaptiveThreshold(a, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, local, 0)
        elif type == 1:
            img = cv2.threshold(a, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
        elif type == 2:
            local = 7
            th1 = cv2.adaptiveThreshold(a, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, local, 0)
            th2 = cv2.threshold(a, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)[1]
            img = th1 & th2
        elif type == 3:
            img = cv2.threshold(a, 60, 255, cv2.THRESH_BINARY)[1]
        ft=FFT(img)

    if len(img.shape)==3:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    sn,_ = ft.shape
    ft_sort=np.unravel_index(np.argsort(-abs(ft), axis=None),ft.shape)
    nft=np.zeros([kernel_size,kernel_size],dtype=complex) 
    nft[ft_sort[0][1:2*(max+1)],ft_sort[1][1:2*(max+1)]]=ft[ft_sort[0][1:2*(max+1)],ft_sort[1][1:2*(max+1)]]
    tmp = np.abs(nft)*unit_circle(sn//2, rrange) 

    if tmp.argmax()==0:   # that there's no pattern at all
        return([-1, -1, -1])     
    dx, dy = np.unravel_index(tmp.argmax(), tmp.shape)
    nft=np.zeros([kernel_size,kernel_size],dtype=complex) 
    nft[dx, dy]=ft[dx, dy]
    nft[kernel_size-dx, kernel_size-dy]=ft[kernel_size-dx, kernel_size-dy]
    if plot=="v":
        plt.figure()
        plt.subplot(311)
        plt.imshow(img,**plot_args)
        plt.subplot(312)
        plt.imshow(np.abs(nft))
        plt.subplot(313)
        plt.imshow(iFFT(nft))    
    elif plot=="h":
        plt.figure()
        plt.subplot(131)
        plt.imshow(img, **plot_args)
        plt.subplot(132)
        plt.imshow(np.abs(nft))
        plt.subplot(133)
        plt.imshow(iFFT(nft))
    x = img.flatten()
    y = iFFT(nft).flatten()

    # all of these three lie in -1 to 1
    if corr == "Pearson":
        conf = scipy.stats.pearsonr(x, y)[0]
    elif corr == "Spearman":
        conf = scipy.stats.spearmanr(x, y).statistic
    elif corr == "Kendall":
        conf = scipy.stats.kendalltau(x, y).statistic
    degree = np.arctan((dy-sn//2)/(dx-sn//2+0.001))
    if degree <= 0:
        degree +=np.pi
    dist = kernel_size/np.sqrt((dy-sn//2)**2+(dx-sn//2)**2)
    degree = degree/np.pi*180
    return([conf, degree, dist])





rendered = cv2.imread("../data/rendered/Tcell/A1.png")
test =cv2.cvtColor(rendered, cv2.COLOR_BGR2GRAY)
test1=test[1300:2300,1400:2200]

# rendered = cv2.imread("/Users/suestring/Desktop/Lab/Code/Hongyun/data/rendered/NA2.png")
# test =cv2.cvtColor(rendered, cv2.COLOR_BGR2GRAY)
# test1=test[1:1500,1:1500]

#Default unit: nm
pixel_size=16
sx, sy=test1.shape
l_ratios=[ (x*180/64/pixel_size) for x in [4,5]]
spectrin_range=np.array([200, 140])  
kernel_size = 64
rrange=(kernel_size*pixel_size/spectrin_range).astype(int)
rrange[1]+=1

for ratio in l_ratios:
    t_test = cv2.resize(test1, (int(sy/ratio), int(sx/ratio)))
    t_pixel = pixel_size*ratio
    rrange=(kernel_size*t_pixel/spectrin_range).astype(int)
    rrange[1]+=1
    corr="Pearson"
    for ntype in range(4):
        c, o, d = FFTfilter(t_test, rrange, kernel_size=64, threshold=0.2, sparse=100000, steps=8, method="bin", type=ntype)
        plt.figure()
        a=sumMat(t_test,c,steps=8, method="gaussian")
        plt.imshow(cv2.merge([t_test,(255*a/np.max(a,axis=None)).astype(np.uint8),t_test]))


In [ ]:
rendered = cv2.imread("/Users/suestring/Desktop/Lab/Code/Hongyun/data/rendered/A1.png")
test =cv2.cvtColor(rendered, cv2.COLOR_BGR2GRAY)
test1=test[1300:2300,1400:2200]

space=[5,8]

plt.figure()
plt.imshow(test1)
xylist=[[430, 290],[600,400],[570,390]]
for x, y in xylist:
    plt.gca().add_patch(Rectangle((y,x),64,64,linewidth=1,edgecolor='r',facecolor='none'))

for x, y in xylist:
    test2=test1[x:x+64, y:y+64]
    plt.figure()
    plt.imshow(test2)
    space=[5,8]
    corr="Pearson"
    print(fft_filter(test2, space, plot="h"))


c1,o1,d1 =FFTfilter(test1, space, kernel_size=64, threshold=0, sparse=200000, steps=8)


sx, sy=test1.shape
nc=np.array(Image.fromarray(c1).resize([sy, sx]))
plt.figure(figsize=[10,5])
plt.subplot(121)
plt.imshow(test1)

plt.subplot(122)
plt.imshow(test1)
plt.imshow(nc, vmin=0.1, vmax=0.7, alpha=0.3,cmap="hsv")

In [ ]:
n_size=test.shape
print(test.shape)
print(c1.shape)
nc=np.array(Image.fromarray(c1).resize([1316,928]))
plt.imshow(nc)

In [ ]:
test.shape

## Simulate complex pattern

In [ ]:
cn=5
sim_complex = np.zeros([cn*64,cn*64])
space = 9
for i in range(cn):
    for j in range(cn):
        theta=np.random.randint(0,180)
        k = np.random.randint(3)
        #theta = 90
        #k = 1
        if k == 0:    #random
            simTmp = simRand(space=13)
        elif k == 1:
            simTmp = sim1D()
        elif k == 2:
            simTmp = sim2D(space=11)
        elif k == 3:
            simTmp = sim2DC(space=11)    
        sim_tmp=genPic(np.zeros([128,128]), simTmp, 9, theta, rand=2, randz=5)
        sim_complex[i*64:(i+1)*64, j*64:(j+1)*64] = sim_tmp[32:96,32:96]


plt.imshow(sim_complex)

space = [6,10]
c, o, d =FFTfilter(sim_complex, space, kernel_size=64, threshold=0, sparse=0, steps=8,plot=1)

In [ ]:
p_size = 16
thre = 190/p_size
nx = ny = 10
space=8
for shift in [0,3,5]:
    points=[]
    for py in range(1, 128, int(thre)):
        for px in range(1+py//int(thre)*shift, 128, space):
            points.append([px,py])
    
    for theta in [0,17,129]:
        sim_1D = genPic(np.zeros([128,128]), points, z, theta, rand=2)
        ft=FFTplot(sim_1D[32:96,32:96], plot=0)
        _=mainFFT(ft,3)
        #plt.figure()
        #plot_max3_r(ft,20)

### Reconstruct 2D and 2DC

In [ ]:
def add6(ft, real, imag, x, trans=0):
    h = int(x*np.sqrt(3))
    x = int(x)
    ft[32+x,32+h]=-real-imag*1j
    ft[32-x,32-h]=-real+imag*1j
    ft[32+2*x,32]=-real+imag*1j
    ft[32+x,32-h]=-real-imag*1j
    ft[32-2*x,32]=-real-imag*1j
    ft[32-x,32+h]=-real+imag*1j
    if trans==1:
        ft=ft.T
    return(ft)

x = 6
t1=add6(np.zeros([64,64],dtype=complex), -2, 0, x)
t2=add6(np.zeros([64,64],dtype=complex), -1.4, 0, x//2)
t3=add6(np.zeros([64,64],dtype=complex), -1.4, 0, x*np.sqrt(3)/2, trans=1)
_=iFFTplot(t1, angle="h", xylen=20, cmap="PuOr")
_=iFFTplot(t2, angle="h", xylen=20, cmap="PuOr")
_=iFFTplot(t3, angle="h", xylen=20, cmap="PuOr")
_=iFFTplot(t1-t2, angle="h", xylen=20, cmap="PuOr")
_=iFFTplot(t1-t2-t3, angle="h", xylen=20, cmap="PuOr")

In [ ]:
# fil = np.array([[  0, 1, 1, 1, 0],                        #这个是设置的滤波，也就是卷积核
#                 [  1, 2, 2, 2, 1],
#                 [  1, 2, 3, 2, 1],
#                 [  1, 2, 2, 2, 1],
#                 [  0, 1, 1, 1, 0]])

fil = np.array([[  0, 1, 0],                        #这个是设置的滤波，也就是卷积核
               [  1, 0, 1],
               [  0, 1, 0]])

t1=add6(np.zeros([64,64],dtype=complex), -2, 0, x)
_=iFFTplot(t1, angle="h", xylen=20, cmap="PuOr")  
res = cv2.filter2D(np.abs(t1),-1,fil)    
_=iFFTplot(res, angle="h", xylen=20, cmap="PuOr")
t1=np.zeros([64,64],dtype=complex)
t1[30,34]=1
res = cv2.filter2D(np.abs(t1),-1,fil)  
_=iFFTplot(t1, angle="h", xylen=20, cmap="PuOr")  
_=iFFTplot(res, angle="h", xylen=20, cmap="PuOr")



In [ ]:

t1=np.zeros([64,64],dtype=complex)
# for i in [1,5,20]:
#     t1[31,35]=1+i*j
#     plt.figure()
ft=FFTplot(test2, plot=0)
plt.subplot(131)
plt.imshow(np.abs(ft))
plt.subplot(132)
plt.imshow(np.real(ft))
plt.subplot(133)
plt.imshow(np.imag(ft))

plt.figure()
plt.subplot(131)
plt.imshow(abs(iFFT(np.abs(ft))))
plt.subplot(132)
plt.imshow(abs(iFFT(np.real(ft))))
plt.subplot(133)
plt.imshow(abs(iFFT(np.imag(ft))))

plt.figure()
plt.subplot(131)
plt.imshow(abs(iFFT(ft)))
plt.subplot(132)
plt.imshow(np.real(iFFT(ft)))
plt.subplot(133)
plt.imshow(abs(iFFT(np.imag(ft))))

In [ ]:
for i in range(8):
    test=cv2.imread(test_1d[i])
    test =cv2.cvtColor(test, cv2.COLOR_BGR2GRAY)
    plt.figure()
    plt.subplot(131)
    plt.imshow(test)
    plt.subplot(132)
    plt.imshow(signal.fftconvolve(test,np.flip(test), mode = 'same'))
    plt.subplot(133)
    plt.imshow(signal.fftconvolve(test,test, mode = 'same'))

In [ ]:
plt.subplot(131)
plt.imshow(test)
plt.subplot(132)
plt.imshow(signal.correlate2d(test,np.flip(test), mode = 'same'))
plt.subplot(133)
plt.imshow(signal.fftconvolve(test,np.flip(test), mode = 'same'))

In [ ]:
test=cv2.imread(test_1d[7])
test =cv2.cvtColor(test, cv2.COLOR_BGR2GRAY)
a=signal.fftconvolve(test,np.flip(test), mode = 'same')
local=7
tmp = 255* a.astype(np.float64) / a.max()
a = tmp.astype(np.uint8)
ret1, th1 = cv2.threshold(a, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)
th2 = cv2.adaptiveThreshold(a, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, local, 0)
th3=th1&th2

plt.subplot(131)
plt.imshow(test)
plt.subplot(132)
plt.imshow(a)
plt.subplot(133)
plt.imshow(th2)

In [ ]:
print(fft_filter(test, [5,7], plot="h"))
print(fft_filter(a, [5,7], plot="h"))
print(fft_filter(th2, [5,7], plot="h"))
#def fft_filter(img, rrange, max=6, plot=0, kernel_size=64, corr="Pearson", method="fft", **plot_args):


In [ ]:
print(fft_filter(test, [5,7], plot="h", method="rb_auto"))


In [ ]:

print(fft_filter(test, [5,7], plot="h", method="ostu"))


In [ ]:
for i in range(8):
    test=cv2.imread(test_2d[i])
    test =cv2.cvtColor(test, cv2.COLOR_BGR2GRAY)
    plt.figure()
    print(fft_filter(test, [5,7], plot="h", method="ostu"))


In [ ]:
psize = 9
theta = 56
points=simRand()
np.random.shuffle(points)  
l=[[] for x in range(5)]
methods = ["fft","auto","bin","bin1"]

for i in range(20):
    sim_Rand = genPic(np.zeros([128,128]), points[:i*19+20,:], psize, theta, rand=0, randz=1)
    plt.figure()
    plt.imshow(sim_Rand[32:96,32:96])
    # for j in range(4):
    #     method = methods[j]
    #     l[j].append(fft_filter(sim_Rand[32:96,32:96], [5,7], vmin=-10, vmax=800, method=method)[0])

In [ ]:
for i in range(4):
    plt.plot(l[i])

plt.legend(methods)

In [ ]:
ft=FFT(sim_1D[32:96,32:96])
sn,_ = ft.shape
ft_sort=np.unravel_index(np.argsort(-abs(ft), axis=None),ft.shape)
nft=np.zeros([kernel_size,kernel_size],dtype=complex) 
nft[ft_sort[0][1:2*(max+1)],ft_sort[1][1:2*(max+1)]]=ft[ft_sort[0][1:2*(max+1)],ft_sort[1][1:2*(max+1)]]
tmp = np.abs(nft)*unit_circle(sn//2, rrange) 
dx, dy = np.unravel_index(tmp.argmax(), tmp.shape)
nft=np.zeros([kernel_size,kernel_size],dtype=complex) 
nft[dx, dy]=ft[dx, dy]
nft[kernel_size-dx, kernel_size-dy]=ft[dx, dy]
iFFTplot(nft)

In [ ]:
psize = 9
theta = 56
for x in range(10):
    points=simRand()
    np.random.shuffle(points)  
    l_r = []
    for i in range(20):
        sim_Rand = genPic(np.zeros([128,128]), points[:i*19+20,:] , psize, theta, rand=0, randz=2)
        l_r.append(scipy.stats.pearsonr(iFFT(nft).flatten(), sim_Rand[32:96,32:96].flatten())[0])
    plt.plot(l_r)
plt.plot(l_1[1:])

In [ ]:
455/19

In [ ]:
points.shape

In [ ]:
np.vstack((points,np.random.randint(32, 96,[nRand,2])))